In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM
from transformers import AutoConfig
import torch
import torch.nn as nn
from gpt2_patch import replace_gpt2_attn_with_flash_attn

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('ai-forever/ruGPT-3.5-13B')

In [3]:
replace_gpt2_attn_with_flash_attn()

In [4]:
path = 'experiments_wiki_4096_4bit/'

In [5]:
#rugpt_config = AutoConfig.from_pretrained('ai-forever/ruGPT-3.5-13B')
#rugpt_config.n_positions = 8192
#rugpt_config.use_cache = False
model = AutoModelForCausalLM.from_pretrained(
    'ai-forever/ruGPT-3.5-13B',
    #config=rugpt_config,
    torch_dtype=torch.float32,
    load_in_4bit=True,
    device_map='auto',
    #ignore_mismatched_sizes=True
)
model.config.n_positions = 4096
model.config.use_cache = False

Loading checkpoint shards: 100%|██████████| 6/6 [01:35<00:00, 15.98s/it]


In [6]:
mini_text = \
"""В ворота гостиницы губернского города NN въехала довольно красивая рессорная небольшая бричка, в какой ездят холостяки: отставные подполковники, штабс-капитаны, помещики, имеющие около сотни душ крестьян, — словом, все те, которых называют господами средней руки. В бричке сидел господин, не красавец, но и не дурной наружности, ни слишком толст, ни слишком тонок; нельзя сказать, чтобы стар, однако ж и не так чтобы слишком молод. Въезд его не произвел в городе совершенно никакого шума и не был сопровожден ничем особенным; только два русские мужика, стоявшие у дверей кабака против гостиницы, сделали кое-какие замечания, относившиеся, впрочем, более к экипажу, чем к сидевшему в нем. «Вишь ты, — сказал один другому, — вон какое колесо! что ты думаешь, доедет то колесо, если б случилось, в Москву или не доедет?» — «Доедет», — отвечал другой. «А в Казань-то, я думаю, не доедет?» — «В Казань не доедет», — отвечал другой. Этим разговор и кончился. Да еще, когда бричка подъехала к гостинице, встретился молодой человек в белых канифасовых панталонах, весьма узких и коротких, во фраке с покушеньями на моду, из-под которого видна была манишка, застегнутая тульскою булавкою с бронзовым пистолетом. Молодой человек оборотился назад, посмотрел экипаж, придержал рукою картуз, чуть не слетевший от ветра, и пошел своей дорогой.
Когда экипаж въехал на двор,"""

mega_text = \
"""Дворцо́вая пло́щадь (с 1918 до 1944 — площадь Урицкого) — главная площадь Санкт-Петербурга, архитектурный ансамбль, возникший во второй половине XVIII — первой половине XIX века.

Площадь образуют памятники истории и культуры федерального значения: Зимний дворец, Здание штаба Гвардейского корпуса, Здание Главного штаба с Триумфальной аркой, Александровская колонна. Её размеры составляют около 5,4 га; существует также неверная оценка в 8 га. В составе исторической застройки центра Санкт-Петербурга площадь включена в список всемирного наследия и является пешеходной зоной.

Во время праздничных мероприятий на площади может собраться до 35 тысяч человек (на «Алых парусах» в 2016 году).

История названия 
Первоначальное название Адмиралтейский луг (включал также территорию современного Александровского сада) известно с 1736 года. Дано по Адмиралтейской верфи. Наименование существовало до 1772 года.

Наименование Дворцовая площадь известно с 1766 года. Дано по расположенному рядом Зимнему дворцу, южный фасад которого выходит на площадь. В октябре 1918 года площадь была переименована в площадь Урицкого (в честь М. С. Урицкого, одного из организаторов штурма Зимнего дворца в 1917 году, председателя Петроградской ЧК, убитого 30 августа 1918 года при входе в здание Главного штаба). 13 января 1944 года вышло постановление о возвращении 20 исторических наименований, в том числе и Дворцовой площади.

История

Предыстория площади 

Предыстория площади связана с закладкой 5 ноября 1704 года Адмиралтейской крепости-верфи. Согласно требованиям военного времени Адмиралтейство окружили валами и рвом. Перед ним простиралось обширное открытое пространство — гласис, необходимый для действий крепостной артиллерии в случае вражеского нападения с суши. Вскоре после основания Адмиралтейство утратило функцию боевой крепости и вместе с ней постепенно ушло в прошлое и фортификационное значение гласиса. Первое время его территорию использовали для складирования и хранения строительного корабельного леса, больших якорей, других адмиралтейских припасов. Примерно с 1712 по 1717 год на части бывшего гласиса находился Морской рынок
, а территория заросла травой и превратилась в Адмиралтейский луг.

В 1721 году по инициативе Петра I была заложена основная планировочная схема Санкт-Петербурга в виде трелучника, исходящего от Адмиралтейства. Два луча (нынешние Невский и Вознесенский проспекты) возникли при Петре I, а третий луч (современная Гороховая улица) появился в 1736—1737 годах. Лучи этих трёх магистралей разделили огромный Адмиралтейский луг на несколько частей. Со времени царствования Анны Иоанновны в этом месте за казённый счёт устраивались празднества с фейерверками и народными гуляньями. На лугу на время торжеств по высочайшим указам возводили потешные павильоны, дворцы, устраивали винные фонтаны, жарились гигантские туши быков, которые затем отдавались на съедение народу. Сохранился любопытный документ от 29 апреля 1743 года, когда Елизавета повелела: «…луг против Зимнего дворца разровнять и засеять овсом». До 1760-х годов Адмиралтейский луг служил вспомогательной строительной площадкой императорского Зимнего дворца. В промежутках между перестройками дворца луг использовался для строевых учений воинских частей и выпаса придворного скота.

В 1753 году Франческо Растрелли предложил архитектурно организовать растекающуюся огромную территорию, создать центральную площадь столицы. Одновременно с окончательным вариантом Зимнего дворца Растрелли проектировал площадь напротив него. В центре площади, по оси здания, архитектор предполагал установить конный монумент Петра I работы своего отца. Статуя должна была находиться внутри колоннады, состоящей из двух колец с четырьмя прорывами, и бо́льшим из них в сторону императорского дворца. Части луга, оказавшиеся за пределами колоннады, Растрелли намеревался решить в виде куртин строгой геометрической формы. Проект площади получил замкнутый, обособленный характер. Колоннада не была увязана с планировкой примыкающих частей города. Однако центральная ось площади (север — юг), определённая зодчим, прошедшая через главные ворота дворца, выведенная в Большую Луговую, сохранена во всех последующих проектах и окончательно закреплена К. Росси.

В 1763 году Комиссией о каменном строении была опубликована конкурсная программа на составление генерального плана города с целью его дальнейшего благоустройства. Итогом конкурса стал план реконструкции центральной — Дворцовой площади Петербурга, выполненный Алексеем Квасовым и утверждённый 8 февраля 1765 года. Застройка с восточной стороны площади, согласно заданию, была сохранена, а западную границу площади архитектор наметил симметрично направлению Луговой, причём прямая линия этой границы на севере приобретала закругление. Эта линия сохранена на всех планах последующего времени.

16 июня 1766 года на Дворцовой площади состоялась знаменитая Петербургская Карусель, отличавшаяся особой роскошью, фантазией и размахом. Участники Карусели делились на четыре кадрили: Римскую, Славянскую, Турецкую и Индийскую. Каждая группа имела соответствующие наряды, лошадиную сбрую, вооружение, особые колесницы для участвующих в состязании дам и даже различные музыкальные инструменты — всё это делали специально для праздника. По проекту Антонио Ринальди на площади, которая в то время представляла поросший травой луг, были построены пятиярусный деревянный амфитеатр на несколько тысяч зрителей с отдельными ложами для Екатерины II и двенадцатилетнего великого князя Павла Петровича. Поверху шла балюстрада, украшенная вазами, а барьер расписан гирляндами, воинскими доспехами, львиными головами. Участники Карусели были в роскошных туалетах. Зрители увидели переливающуюся гору богатства и изобилия в драгоценных камнях и кавалерских и конных золотых и серебряных уборах, ценой в многие миллионы рублей. Победителями среди дам стала графиня Наталья Чернышёва, а среди кавалеров — Григорий Орлов в уборе римского воина на гнедом коне.

Возникновение и развитие площади 
Основой застройки Дворцовой площади стал существующий Зимний дворец (1754—1762, архитектор Б. Ф. Растрелли). Обращённый к Дворцовой площади южный фасад прорезан трёхчастной аркой. В период с 1764 по 1775 год в удалённых концах площади возникают здания Вольного экономического общества на углу с Невским пр. (арх. Валлен-Деламот, перестроено) и Южный павильон Малого Эрмитажа (арх. Фельтен).

20 декабря 1778 года императрица велит преобразовать площадь «против императорского Зимнего дворца, между восточным углом Адмиралтейства и Миллионною улицею». В 1779 году Академия художеств объявляет конкурс на застройку площади. По итогам конкурса предпочтение было отдано модели «с порталом дорического ордера и антаблементом ордера коринфского… по открытии записки, приобщённой при упомянутой модели», сочинителем её оказался архитектор Ю. М. Фельтен. Предложение Фельтена фактически явилось типовым проектом по застройке площади, согласно плану реконструкции, разработанному Алексеем Квасовым ещё в 1765 году. Возведённые в 1779—1784 годах здания — «Глазовский дом», дом А. Д. Ланского, дом Я. А. Брюса — выстраивались по условной дуге.

«§ 113. Дворцовая площадь перед зимним дворцом окружена построенными ИМПЕРАТРИЦЕЮ в 1788 году тремя домами наподобие амфитеатра. На одной сделаны две беседки с загородками и скамьями из дикого камня под железною кровлею, в которых зимою раскладывается огонь для кучеров, стоящих на улице во время съезда при дворе. На сей площади сменяется гвардия, вступающая на караул, и при великих торжествованиях даются здесь народу жареные быки и фонтаны с вином. В торжественные дни от гвардии и прочих команд производятся на оной поздравления музыкою и барабанным боем.» 1794 год.

В 1810 году Александр I поручил архитектору Строительной комиссии Императорского кабинета А. А. Модюи разработку проекта регулирования и оформления территории перед императорской резиденцией — Зимним дворцом, Адмиралтейского луга и Исаакиевской площади. Представленные проекты были одобрены, однако началу реализации проектов помешало изменение международной обстановки. К реализации проекта реконструкции вернулись после окончания военных действий.

В 1816 году архитектором А. Модюи представлен проект перепланировки центральных площадей, на котором Дворцовая площадь получает известные нам очертания: застройка восточной части симметрична западной.

Весной 1819 года императором Александром I было принято решение о придании площади перед Зимним дворцом вида, подобающего центру столицы Российской империи. Реконструкцией занимался Комитет строений и гидравлических работ. Руководитель Комитета А. Бетанкур привлёк к участию в работах Карла Росси. 16 марта 1819 года Росси получил распоряжение разработать проект реконструкции старых зданий для Главного штаба. Через год, 15 марта 1820 года был издан указ о создании в составе Строительной комиссией Императорского кабинета особого стола «По устроению против Зимнего дворца правильной площади и каменным, кирпичным, гончарным и известковым заводам» с участием Росси. Стол находился в непосредственном подчинении Бетанкура.

Получив распоряжение разработать проект здания, определяющего «устройство против Зимнего дворца правильной площади», Росси не ограничился приспособлением старых домов для нового использования. В 1819—1829 годах фельтеновские дома с их основными пропорциями и дугообразным положением в плане вошли в ансамбль здания Главного штаба (архитектор К. И. Росси). Оба крыла здания Росси соединяет триумфальной аркой. Два корпуса, соединённые аркой, охватывают пространство площади, придают её облику торжественность и монументальность.
Росси решил градостроительную задачу, создав композицию в рамках сложного плана участка, заданной ещё Квасовым дугообразной линии границы площади и острого угла площади с рекой Мойкой

В 1837—1843 годах с восточной стороны Дворцовой площади на месте Экзерциргауза (1797—1798, архитектор В. Бренна) по проекту архитектора А. П. Брюллова построено большое здание штаба Гвардейского корпуса. В 1830—1834 годах в центре площади по проекту архитектора Огюста Монферрана была возведена Александровская колонна (фигура ангела выполнена скульптором Б. И. Орловским). Как и Триумфальная арка Главного штаба, памятник посвящён победам русского оружия в войне с Наполеоном.

С юго-западной стороны до 1840-х годов на углу Дворцовой площади и Невского проспекта находилось здание Вольного экономического общества. В 1845—1846 годах на этом месте построено здание (), фасад которого гармонично сочетается с фасадом Главного штаба.
С северо-западной стороны, между Адмиралтейством и Зимним дворцом до конца XIX века существовала Разводная площадь. Площадь использовалась для развода караула, была образована после уничтожения канала и бастионов Адмиралтейской крепости. В 1896—1901 годах на её месте перед дворцом был разбит сквер с фонтаном (архитекторы Н. И. Крамской, Р. Шмеллинг, садовод Р. Ф. Катцер). В 1920-х годах ограда сквера была демонтирована и использована при украшении парка им. 9 Января на проспекте Стачек. Сам фонтан был реконструирован в 2007—2008 годах с восстановлением исторического вида.

В ΧΙΧ — начале ΧΧ веков Дворцовая площадь была местом проведения военных смотров и парадов. Девятого января 1905 года на Дворцовой площади была расстреляна царскими войсками демонстрация рабочих. В память об этом событии Дворцовая набережная была переименована в Набережную 9-го января (до 1944). В ночь с 25 на 26 октября (7—8 ноября) 1917 года на Дворцовой площади произошёл решающий бой Октябрьского вооружённого восстания в Петрограде.

В начале XX века Манежная площадь находилась на территории Адмиралтейской полицейской части.

Перед Первой мировой войной все здания на Дворцовой площади были окрашены в красно-кирпичные цвета, на фоне которых и проходили события 1917 года. В 1940-х годах здания снова перекрасили в присущие им светлые тона.

Советское время 
В советское время Дворцовая площадь была местом проведения демонстраций и парадов в дни революционных праздников. С октября 1918 года Дворцовая площадь официально называлась площадью Урицкого (в честь М. С. Урицкого, убитого в здании Главного штаба).

22 сентября 1918 года на Дворцовой площади был установлен гипсовый памятник А. Н. Радищеву по проекту скульптора Л. В. Шервуда, который стал первым памятником, установленным согласно плану монументальной пропаганды, принятому ленинским правительством 14 апреля 1918 года. Бюст опрокинуло ветром в январе 1919 года, и восстанавливать его не стали.

В 1918—1921 году на площади проходили масштабные театрализованные представления: «Действо о III Интернационале», «Мистерия освобождённого труда», «К мировой коммуне», «Взятие Зимнего дворца» (постановщики — Николай Евреинов, Юрий Анненков и другие). 7 ноября 1920 года на площади Урицкого перед ста тысячами зрителей были разыграны сцены, изображавшие события Октябрьской революции. В представлении принимали участие 6 тысяч человек.

11 ноября 1924 года, в честь празднования Октябрьского переворота, был издан правительственный указ о реконструкции Александровской колонны — снять фигуру ангела и заменить её на скульптуру Ленина. Специальная комиссия в течение года рассматривала различные варианты изменения колонны, однако в конце концов от этого проекта отказались.

20 июля 1924 года на площади состоялось представление «живых шахмат» с участием бойцов Красной армии (белые) и ВМФ (чёрные); играли шахматные мастера Илья Рабинович и Пётр Романовский.

В 1932 году проводится реконструкция площади, в ходе которой брусчатку заменили на асфальт, а также убрали гранитные столбики вокруг Александровской колонны.

С 1935 года на площади проходит финиш пробега Пушкин-Ленинград, старейшего пробега СССР.

В годы Великой Отечественной войны, осенью 1941 года рассматривался вариант размещения на площади аэродрома для истребительного авиационного полка, при этом должна была быть перенесена Александровская колонна и вырублен Адмиралтейский сад, однако от этой идеи отказались.

Историческое название было восстановлено 13 января 1944 года. В 1977 году на Дворцовой площади были проведены значительные реконструктивные работы и выполнено декоративное мощение, асфальтовое покрытие заменено диабазовой брусчаткой, воссозданы в первоначальных формах четыре фонаря по её углам.

20 августа 1991 года на Дворцовой площади прошёл стихийный митинг в знак протеста против действий ГКЧП СССР. В митинге участвовало около ста тысяч человек.

Современный период 

В ходе реставрации Дворцовой площади в 2001 году производились археологические исследования. В результате раскопок был найден фундамент флигеля Анны Иоанновны. В своё время он представлял собой трёхэтажный дворец, возведённый по проекту Растрелли в 1746 году. В соответствии с планом реставрации площади фундамент был изучен, сфотографирован и вновь зарыт землёй. В качестве одного из вариантов рассматривалась идея оставить фрагмент фундамента для обозрения, закрыв его толстым прочным стеклом.

В 2006 году была выполнена реставрация Александровской колонны.

На Дворцовой площади с 1990-х годов проводятся концерты, ежегодные спортивные и общественные мероприятия:
 Парад, посвящённый Дню города.
 Ежегодные международные легкоатлетические пробеги, такие как Пробег Пушкин — Санкт-Петербург и марафон «Белые ночи».
 Неофициальный праздник любителей бардовской песни проходит ежегодно 1 сентября у стен Зимнего дворца.
 Концерты музыкантов. Сборный концерт во время Августовского путча ГКЧП «Рок против танков» (22 августа 1991 года); группа «ДДТ» (27 мая 1993 года); Джо Кокер (15 сентября 2005 года); оперный тенор Пласидо Доминго (16 июня 2001 года); Пол Маккартни (20 июня 2004 года); Элтон Джон (6 июля 2007 года); группа «The Rolling Stones» (28 июня 2007 года); группа «Scorpions» (13 июня 2007 года); Эдита Пьеха (4 августа 2007 года); группа «Машина Времени» (23 сентября 2007 года); Роджер Уотерс (6 июня 2008 года); группа «Duran Duran» (4 июня 2009 года); Анастейша (4 июня 2009 года); Мадонна (2 августа 2009 года).

В ноябре 2007 года на Дворцовой площади был сооружён платный каток с искусственным льдом. Каток сразу же привлёк к себе внимание многочисленных общественных организаций города, которые протестовали против строительства катка, ограничивающего доступ к части Дворцовой площади и подход к Александровской колонне, доступ к которым должен быть свободным. Каток был закрыт в марте 2008 года.

В 2005 году Алексей Ковалёв представил Законодательному собранию Санкт-Петербурга законопроект «Об использовании Дворцовой площади при проведении массовых мероприятий». Предполагалось определить территориальные границы Дворцовой площади и установить правила по регулированию использования Дворцовой площади при проведении массовых мероприятий. Однако законопроект не был поддержан собранием. В 2008 году после истории с катком было объявлено, что Всемирный клуб петербуржцев собирается разработать законопроект о статусе Дворцовой площади и допустимых параметрах её использования.

В сентябре 2015 года на Дворцовой площади был установлен Зеркальный куб — зеркальный павильон кубической формы, в стенах которого отражался весь ансамбль площади: Зимний дворец, Александровская колонна и арка Главного штаба. В зеркальном павильоне прошло частное мероприятие — презентация новой модели BMW. Через некоторое время павильон обрушился из-за сильного ветра и был демонтирован.

Схема площади

Ансамбль площади

Формирование архитектурного ансамбля 
Дворцовая площадь представляет единый архитектурный ансамбль. Северной границей является фасад Зимнего дворца, южная граница имеет полуциркульное очертание, образованное зданием Главного штаба, два трёхэтажных корпуса которого соединены триумфальной аркой, увенчанной колесницей победы. В центре установлена Александровская колонна. С восточной стороны площадь обрамляет здание Штаба гвардейского корпуса.

Постройка Б. Ф. Растрелли в 1754-1762 году Зимнего дворца положила начало архитектурному ансамблю Дворцовой площади. Огромный вклад в формирование архитектурного облика внёс К. И. Росси, доведя ансамбль до логического завершения. Возведение Росси в 1819—1829 годах здания Главного штаба с триумфальной аркой подчеркнуло парадный характер центральной площади Санкт-Петербурга. Классические колоннады фасадов нового здания образовали композиционное единство с барочными формами Зимнего дворца. Особенно эффектно было рассчитано Росси восприятие фасада дворца со стороны Большой Морской улицы, ведущей к арке Главного штаба. Формирование ансамбля завершили сооружение в её центре Александровской колонны по проекту О. Монферрана и возведение здания Штаба Гвардейского корпуса по замыслу А. П. Брюллова.

Александровская колонна 

Воздвигнута в стиле ампир в 1834 году в центре Дворцовой площади французским архитектором Огюстом Монферраном по указу императора Николая I в память о победе его старшего брата Александра I над Наполеоном в Отечественной войне 1812 года.

Проект был высочайше утверждён в 1829 году, монумент был открыт в 1834 году.
Эскизные рисунки пьедестала выполнил О. Монферран. Барельефы на пьедестале колонны в аллегорической форме прославляют победу русского оружия и символизируют отвагу российской армии; отливка была выполнена на заводе Ч. Берда. Монолит розового гранита был изготовлен в Пютерлакской каменоломне под Выборгом в 1830—1832 годах. С большими сложностями он был перевезён из Пютерлакской каменоломни в 1832 году на специально сконструированной для этой цели барже в Санкт-Петербург. Монолит не закреплён и держится только силой собственного веса. Памятник венчает фигура ангела работы Бориса Орловского. В 1876 году архитектором К. К. Рахау были добавлены декоративные фонари у колонны.

Колонна — один из известнейших памятников Петербурга.
Нередко её ошибочно называют Александрийский столп, по стихотворению А. С. Пушкина «Памятник»:

Зимний дворец 

Нынешнее здание дворца построено в 1754—1762 годах по проекту архитектора итальянского происхождения Б. Ф. Растрелли в стиле пышного елизаветинского барокко. С 1732 до 2 марта 1917 года на этом месте находилась официальная зимняя резиденция российских императоров. 29 декабря 1837 года в Зимнем дворце случился пожар. Потушить его не могли три дня, всё это время, вынесенное из дворца имущество, было сложено вокруг Александровской колонны. В 1838—1839 годах дворец был восстановлен по проектам архитекторов под руководством В. П. Стасова, А. П. Брюллова и других. С июля по ноябрь 1917 года Зимний дворец служил местом заседания Временного правительства. В 1918 году часть, а в 1922 году всё здание передано Государственному Эрмитажу.

Современное здание в плане имеет форму квадрата со внутренним двором и развитыми угловыми ризалитами; фасады обращены к Неве, Адмиралтейству и Дворцовой площади. Парадность зданию придают пышная отделка фасадов и помещений. Главный фасад, обращённый к Дворцовой площади, прорезан трёхчастной аркой парадного проезда. В юго-восточной части второго этажа находился один из памятников рококо — Большая церковь Зимнего дворца, спроектированная ещё в 1763 году архитектором Растрелли.

Здание штаба Гвардейского корпуса (дома 2—4) 

С востока ансамбль Дворцовой площади замыкает фасад здания бывшего штаба Гвардейского корпуса, построенного в позднеклассическом стиле по проекту архитектора А. П. Брюллова в 1837—1843 годах. На этом месте с конца ΧVΙΙΙ века находилось здание экзерциргауза, возведённое В. Бренной. Оно тянулось вдоль Миллионной улицы. В 1827 году был объявлен конкурс на проект строительства театра рядом с экзерциргаузом с единым «приличным» фасадом. Но затем от планов отказались.

Перед Брюлловым стояла задача замкнуть гигантский периметр, соединив несовместимое — барочное здание Растрелли с классическим зданием Росси. И он нашёл решение, поставив 4-этажное здание, которое и изящно, и гармонично, но при этом нейтрально.

Сейчас в здании Штаба Гвардейского корпуса находится штаб 6-й Ленинградской Краснознамённой армии военно-воздушных сил и противовоздушной обороны.

Здание Главного штаба (дома 6—10) 

С юга площадь обрамляет здание Главного штаба, построенное в стиле ампир по проекту архитектора К. И. Росси в 1810—1829 годах. Здание состоит из трёх корпусов (два в восточной части и одно в западной), составляющих вместе дугу общей длиной 580 метров, соединённых триумфальной аркой, которая является композиционным центром и открывает парадный въезд на площадь со стороны Большой Морской улицы. Венчает арку победная колесница Славы (архитекторы скульптур В. И. Демут-Малиновский и С. С. Пименов).

В зданиях, помимо Главного Штаба, располагались Военное министерство, Министерство иностранных дел и Министерство финансов (в восточном корпусе). После Октябрьской революции в здании располагался Наркомат Иностранных дел, а впоследствии отделение милиции. В настоящее время часть здания принадлежит Западному военному округу. В 1993 году восточное крыло здания Главного штаба передано Эрмитажу.

Транспорт 

Рядом с Дворцовой площадью начиналась история общественного транспорта Санкт-Петербурга: 27 августа 1863 года открылся 1-й маршрут конно-железной дороги от Николаевского (ныне — Московского) вокзала мимо Дворцовой площади до Стрелки Васильевского острова. Затем конки стали ходить от Дворцовой площади до 6-й линии Васильевского острова через Николаевский (ныне — Благовещенский) мост. Первый рейс санкт-петербургского сухопутного трамвая стартовал а от Александровского сада, а 11 ноября оттуда же отправился первый автобус по маршруту Александровский сад — Балтийский вокзал. После большого перерыва при советской власти 24 декабря 1926 года от площади было возобновлено движение автобусов по маршруту площадь Урицкого (Дворцовая площадь) — Загородный проспект — площадь Восстания.

По состоянию на январь 2017 года никакой общественный транспорт через площадь не проходит. Ближайшие станции метро:
"""

remainder = """по-английски две-три клумбы с кустами сиреней и желтых акаций; пять-шесть берез небольшими купами кое-где возносили свои мелколистные жиденькие вершины. Под двумя из них видна была беседка с плоским зеленым куполом, деревянными голубыми колоннами и надписью: «Храм уединенного размышления»; пониже пруд, покрытый зеленью, что, впрочем, не в диковинку в аглицких садах русских помещиков. У подошвы этого возвышения, и частию по самому скату, темнели вдоль и поперек серенькие бревенчатые избы, которые герой наш, неизвестно по каким причинам, в ту же минуту принялся считать и насчитал более двухсот; нигде между ними растущего деревца или какой-нибудь зелени; везде глядело только одно бревно. Вид оживляли две бабы, которые, картинно подобравши платья и подтыкавшись со всех сторон, брели по колени в пруде, влача за два деревянные кляча изорванный бредень, где видны были два запутавшиеся рака и блестела попавшаяся плотва; бабы, казалось, были между собою в ссоре и за что-то перебранивались. Поодаль в стороне темнел каким-то скучно-синеватым цветом сосновый лес. Даже самая погода весьма кстати прислужилась: день был не то ясный, не то мрачный, а какого-то светло-серого цвета, какой бывает только на старых мундирах гарнизонных солдат, этого, впрочем, мирного войска, но отчасти нетрезвого по воскресным дням. Для пополнения картины не было недостатка в петухе, предвозвестнике переменчивой погоды, который, несмотря на то что голова продолблена была до самого мозгу носами других петухов по известным делам волокитства, горланил очень громко и даже похлопывал крыльями, обдерганными, как старые рогожки. Подъезжая ко двору, Чичиков заметил на крыльце самого хозяина, который стоял в зеленом шалоновом сюртуке, приставив руку ко лбу в виде зонтика над глазами, чтобы рассмотреть получше подъезжавший экипаж. По мере того как бричка близилась к крыльцу, глаза его делались веселее и улыбка раздвигалась более и более."""

In [7]:
len(tokenizer.tokenize(mega_text))

Token indices sequence length is longer than the specified maximum sequence length for this model (5201 > 2048). Running this sequence through the model will result in indexing errors


5201

In [8]:
def perplexity(text, model, max_tokens=2048):
    input = tokenizer([text], return_tensors="pt", max_length=max_tokens, truncation=True)
    ids = input['input_ids']
    tgts = ids.clone()
    with torch.no_grad():
        outputs = model(ids, labels=tgts)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss
    return torch.exp(neg_log_likelihood.float().mean())


def generate(text, model, max_new_tokens=50, num_beams=2):
    input = tokenizer([text], return_tensors="pt")
    for k, v in input.items():
        input[k] = v.to(model.device)
    n_input = input['input_ids'].shape[1]
    outputs = model.generate(**input, max_new_tokens=max_new_tokens, num_beams=num_beams, do_sample=True)
    outputs = outputs[:, n_input:]
    return tokenizer.batch_decode(outputs)[0]

In [9]:
# input = tokenizer([mega_text], return_tensors="pt", max_length=2048, truncation=True)
# ids = input['input_ids']
# tgts = ids.clone()

# outputs = model(ids, labels=tgts)

# # loss is calculated using CrossEntropyLoss which averages over valid labels
# # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
# # to the left by 1.
# neg_log_likelihood = outputs.loss

In [10]:
def interpolate_wpe(wpe: nn.Embedding, target_length=4096, mode='linear'):
    new_embed = nn.Embedding(num_embeddings=target_length, embedding_dim=wpe.embedding_dim)
    # Upsample original weights
    weight = wpe.weight.data
    if weight.ndim == 2:
        weight = weight.unsqueeze(0)
        weight = weight.transpose(1, 2)
    weight = nn.functional.interpolate(weight, size=(target_length), mode=mode)
    weight = weight.squeeze(0).transpose(0, 1).contiguous()
    new_embed.weight = nn.Parameter(weight)
    return new_embed

In [11]:
from scipy.interpolate import interp1d
import numpy as np
import torch


def upsample_general(array, n_samples_upsampled, mode):
    n_samples, n_features = array.shape
    upsampled = np.zeros((n_samples_upsampled, n_features), dtype='float32')
    
    # Create original and upsampled indices
    original_indices = np.linspace(0, 1, n_samples)
    new_indices = np.linspace(0, 1, n_samples_upsampled)
    
    for i in range(n_features):
        interpolator = interp1d(original_indices, array[:, i], kind=mode)
        upsampled[:, i] = interpolator(new_indices)
    
    return upsampled


def interpolate_wpe_v2(wpe: nn.Embedding, target_length=4096, mode='linear'):
    new_embed = nn.Embedding(num_embeddings=target_length, embedding_dim=wpe.embedding_dim)
    # Upsample original weights
    device = wpe.weight.data.device
    dtype = wpe.weight.data.dtype
    weight = wpe.weight.data.cpu().to(torch.float16).numpy()
    weight = upsample_general(weight, target_length, mode)
    weight = torch.from_numpy(weight).to(device=device, dtype=dtype)
    new_embed.weight = nn.Parameter(weight)
    return new_embed

def extend(wpe: nn.Embedding, target_length=4096, mode='linear'):
    device = wpe.weight.data.device
    dtype = wpe.weight.data.dtype
    new_embed = nn.Embedding(num_embeddings=target_length, embedding_dim=wpe.embedding_dim, device=device, dtype=dtype)
    # Upsample original weights
    weight = wpe.weight.data.cpu().to(torch.float16).numpy()
    orig_n_ctx = weight.shape[0]
    new_embed.weight.data[:orig_n_ctx] = wpe.weight.data
    new_embed.weight.data[orig_n_ctx:] = torch.randn((target_length - orig_n_ctx, wpe.embedding_dim), dtype=dtype, device=device)
    return new_embed


def random_wpe(wpe, target_length=4096):
    new_embed = nn.Embedding(num_embeddings=target_length, embedding_dim=wpe.embedding_dim)
    # Upsample original weights
    device = wpe.weight.data.device
    dtype = wpe.weight.data.dtype
    weight = torch.rand(target_length, wpe.embedding_dim).to(device=device, dtype=dtype)
    new_embed.weight = nn.Parameter(weight)
    return new_embed

In [12]:
original_wpe = model.transformer.wpe

In [13]:
original_wpe.weight.data.shape

torch.Size([2048, 5120])

In [14]:
#generate(mini_text, model, num_beams=5)

In [15]:
#perplexity(mega_text, model, max_tokens=2048)

In [16]:
mode = 'linear'

In [17]:
new_wpe = interpolate_wpe(original_wpe, target_length=2048 * 2, mode=mode)
model.transformer.wpe = new_wpe

In [18]:
perplexity(mega_text, model, max_tokens=2048*2)

tensor(13.8136)

In [20]:
from peft import PeftModel, prepare_model_for_int8_training

In [19]:
#bit8_model = prepare_model_for_int8_training(model)

In [23]:
#perplexity(mega_text, bit8_model, max_tokens=2048*2)

tensor(3.6989e+08)

In [28]:
import safetensors

In [29]:
with open('./experiments_wiki_4096/adapter_model.safetensors', 'rb') as f:
    adapter = safetensors.deserialize(f.read())

In [30]:
adapter

In [21]:
peft_model = PeftModel.from_pretrained(model, './experiments_wiki_4096_4bit/checkpoint-50',)

In [23]:
perplexity(mega_text, peft_model, max_tokens=2048*2)

tensor(7.2566)

In [18]:
torch.cuda.empty_cache()

In [16]:
perplexity(mega_text, model, max_tokens=2048), perplexity(mega_text, model, max_tokens=2048*2)

(tensor(10.5431, dtype=torch.float32), tensor(10.6465, dtype=torch.float32))

In [17]:
new_wpe = interpolate_wpe_v2(original_wpe, target_length=2048 * 3, mode=mode)
model.transformer.wpe = new_wpe

In [18]:
perplexity(mega_text, model, max_tokens=2048), perplexity(mega_text, model, max_tokens=2048*2), perplexity(mega_text, model, max_tokens=2048*3)

(tensor(31.5373, dtype=torch.float32),
 tensor(28.0499, dtype=torch.float32),
 tensor(32.3483, dtype=torch.float32))

In [31]:
2048*3

6144

In [32]:
new_wpe = random_wpe(original_wpe, target_length=4096)
model.transformer.wpe = new_wpe
perplexity(mega_text, model)

tensor(12302.5107)

In [19]:
print(generate(mega_text, model))